# Welcome

Stefano Camborda La Cruz  
Dr. Nadine Töpfer
  
*Metabolic Systems Interactions*

GOAL: Use a generic plant metabolic model to create a C4 leaf model

Basic concepts and ideas:
- Extending a large-scale metabolic model
- Reproducibility and robustness
- Implementation using COBRApy and [CobraMod](https://github.com/Toepfer-Lab/cobramod/)

*duration: approx. 50min*

### C4 photosynthesis in a nutshell

| <img src="https://miro.medium.com/max/2400/0*BhzQZYPEV8cGj5mZ.png" alt="drawing" width="900"/> | <img src="https://miro.medium.com/proxy/1*duJX24NyLJ5osWdBHU8Kxg.png" alt="drawing" width="900"/> |
|------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------|
Images taken from [Khan Academy](https://www.khanacademy.org/science/biology/photosynthesis-in-plants/photorespiration--c3-c4-cam-plants/a/c3-c4-and-cam-plants-agriculture)

## Creating the C4 leaf model

Load basic Python modules

In [1]:
from pathlib import Path

from cobra.io import read_sbml_model
from cobra import Solution, Model

from cobramod import add_reactions

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


/home/stefano/miniconda3/envs/test/lib/python3.9/site-packages/cobramod/parsing/biocyc.py:66: UserWarning: Biocyc version is not the same in CobraMod. Please inform maintainers.
  warn(


### Preparation

Loading the model and defining directories

In [2]:
# Load main model
backup = read_sbml_model(
    str(Path.cwd().joinpath("model", "PlantCoreMetabolism_v2_0_0.sbml"))
)
# CobraMod essential directory
dir_data = Path.cwd().joinpath("data")

model = backup.copy()

assert backup
assert dir_data.exists()

### General uptake constraints

Using the attribute `Reaction.bounds`

In [3]:
for reaction in model.exchanges:
    print(reaction.id, reaction.reaction)

Ca_tx  <=> CAII_e
Photon_tx  <=> Photon_e
H_tx PROTON_e --> 
Sucrose_tx  <=> SUCROSE_e
H2O_tx  <=> WATER_e
CO2_tx  <=> CARBON_DIOXIDE_e
O2_tx  <=> OXYGEN_MOLECULE_e
Pi_tx  <=> Pi_e
Mg_tx  <=> MGII_e
Nitrate_tx  <=> NITRATE_e
SO4_tx  <=> SULFATE_e
NH4_tx  <=> AMMONIUM_e
K_tx  <=> KI_e
GLC_tx  <=> GLC_e


In [4]:
# Skipped
# ATP bounds
model.reactions.get_by_id("ATPase_tx").bounds = (0, 1000)

# NTT is only active at night
model.reactions.get_by_id("ATP_ADP_Pi_pc").bounds = (0, 0)

model.reactions.get_by_id("Plastoquinol_Oxidase_p").bounds = (0, 0)

In [5]:
# Defining autotrophic conditions
model.reactions.get_by_id("CO2_tx").bounds = (-1000, 1000)
model.reactions.get_by_id("O2_tx").bounds = (-1000, 1000)
model.reactions.get_by_id("H2O_tx").bounds = (-1000, 1000)

model.reactions.get_by_id("NH4_tx").bounds = (0, 0)
model.reactions.get_by_id("Pi_tx").bounds = (0, 1000)
model.reactions.get_by_id("SO4_tx").bounds = (0, 1000)

model.reactions.get_by_id("Sucrose_tx").bounds = (0, 0)
model.reactions.get_by_id("GLC_tx").bounds = (0, 0)

### Creating and adding new reactions

Using `cobramod` to extend the model

In [6]:
# Malate/Pyruvate transporter
add_reactions(
    model=model,
    obj="PYR_MAL_pc, Pyruvate/Malate transporter | "
    + "PYRUVATE_p + MAL_c <-> PYRUVATE_c + MAL_p",
    directory=dir_data,
)

assert model.reactions.get_by_id("PYR_MAL_pc")

Transporter no esta en el model, pero esta en el sistema c4 de plantas

### Creating mesophyll and bundle sheath cells

We use copies of our model and add prefixes to them

In [7]:
c3_model = model.copy()
c4_mesophyll = model.copy()
c4_bundle = model.copy()

In [8]:
# For the mesophyll cell
for dictlist in [
    c4_mesophyll.reactions,
    c4_mesophyll.metabolites,
]:
    for item in dictlist:
        item.id = f"M_{item.id}"

# For the bundle sheath cell
for dictlist in [
    c4_bundle.reactions,
    c4_bundle.metabolites,
]:
    for item in dictlist:
        item.id = f"B_{item.id}"

c4_model = c4_mesophyll.merge(right=c4_bundle)

In [9]:
c4_bundle

Name,PlantCoreMetabolism_v1_3_0
Memory address,0x07f3040fe2a00
Number of metabolites,861
Number of reactions,893
Number of groups,208
Objective expression,1.0*B_Phloem_output_tx - 1.0*B_Phloem_output_tx_reverse_ef7a0
Compartments,"Mitochondrion, Cytoplasm, Biomass, Plastid, Vacuole, Peroxisome, Endoplasmic reticulum, Mitochondrion innermembrane interacting with cristal space, Mitochondrion innermembrane interacting with inter membrane space, Extracellular, Thylakoid, Mitochondrial intermembrane space"


In [10]:
c4_model

Name,PlantCoreMetabolism_v1_3_0
Memory address,0x07f3040fe2310
Number of metabolites,1629
Number of reactions,1786
Number of groups,208
Objective expression,1.0*M_Phloem_output_tx - 1.0*M_Phloem_output_tx_reverse_d57a8
Compartments,"Mitochondrion, Cytoplasm, Biomass, Plastid, Vacuole, Peroxisome, Endoplasmic reticulum, Mitochondrion innermembrane interacting with cristal space, Mitochondrion innermembrane interacting with inter membrane space, Extracellular, Thylakoid, Mitochondrial intermembrane space"


### Testing the new reactions

- Using the `assert` statement
- Confirm existence of reaction in cells

In [11]:
assert c4_model.reactions.get_by_id("M_O2_tx")
assert c4_model.reactions.get_by_id("B_O2_tx")
assert c4_model.reactions.get_by_id("M_THREDEHYD_RXN_p")
assert c4_model.reactions.get_by_id("B_THREDEHYD_RXN_p")

### Adding transport reactions between the mesophyll and bundle sheath cells

- Selecting metabolites
- Creating reactions
- Testing

In [12]:
for identifier in ["MAL_c", "PYRUVATE_c"]:
    add_reactions(
        model=c4_model,
        obj=f"MB_{identifier}, {identifier} bundle sheath/mesophyll cell "
        f"transporter | M_{identifier} <-> B_{identifier}",
        show_imbalance=False,
        directory=dir_data,
    )
    assert c4_model.reactions.get_by_id(f"MB_{identifier}")

### C4-specific constrains

- No CO2 uptake in bundle sheath cells
- Block rubisco activity in the mesophyll
- Enforce NADP-ME decarboxylation pathways (e.g. in maize)

Examples:

In [14]:
c4_model.reactions.get_by_id("B_CO2_tx").bounds = (0, 0)
# Carboxylase
c4_model.reactions.get_by_id(
    "M_RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p"
).bounds = (0, 0)
# Oxygenase
c4_model.reactions.get_by_id("M_RXN_961_p").bounds = (0, 0)

In [15]:
# Force NADP-ME decarboxylation pathway:
# Block all other decarboxylation reactions except NADP_ME in the plastid
c4_model.reactions.get_by_id("B_PEPCARBOXYKIN_RXN_c").bounds = (0, 0)
c4_model.reactions.get_by_id(
    "B_1_PERIOD_1_PERIOD_1_PERIOD_39_RXN_m"
).bounds = (0, 0)
c4_model.reactions.get_by_id("B_MALIC_NADP_RXN_c").bounds = (0, 0)


In [16]:
# Force NADP-ME decarboxylation pathways:
# make alternative decarboxylation routes irreversible
c4_model.reactions.get_by_id("B_CARBAMATE_KINASE_RXN_p").bounds = (
    0,
    1000,
)
c4_model.reactions.get_by_id("M_CARBAMATE_KINASE_RXN_p").bounds = (
    0,
    1000,
)
c4_model.reactions.get_by_id("B_ISOCITDEH_RXN_m").bounds = (0, 1000)
c4_model.reactions.get_by_id("M_ISOCITDEH_RXN_m").bounds = (0, 1000)
c4_model.reactions.get_by_id("B_ISOCITDEH_RXN_c").bounds = (0, 1000)
c4_model.reactions.get_by_id("M_ISOCITDEH_RXN_c").bounds = (0, 1000)
c4_model.reactions.get_by_id("B_ISOCITRATE_DEHYDROGENASE_NAD_RXN_m").bounds = (
    0,
    1000,
)
c4_model.reactions.get_by_id("M_ISOCITRATE_DEHYDROGENASE_NAD_RXN_m").bounds = (
    0,
    1000,
)

In [17]:
# skipped
# Fix malate transport
c4_model.reactions.get_by_id("B_OAA_MAL_pc").bounds = (0, 1000)

c4_model.reactions.get_by_id("B_PYRUVATE_pc").bounds = (0, 0)

c4_model.reactions.get_by_id(
    "M_PYRUVATEORTHOPHOSPHATE_DIKINASE_RXN_c"
).bounds = (0, 0)

In [18]:
# Skipped
# NGAM for C3
const = c3_model.problem.Constraint(
    (0.0049 * c3_model.reactions.Photon_tx.flux_expression + 2.7852)
    - c3_model.reactions.ATPase_tx.flux_expression,
    lb=0,
    ub=0,
)
c3_model.add_cons_vars(const)

# ATP/NADPH 3:1 constraints
const = c3_model.problem.Constraint(
    c3_model.reactions.ATPase_tx.flux_expression
    - 3
    * (
        c3_model.reactions.NADPHoxc_tx.flux_expression
        + c3_model.reactions.NADPHoxp_tx.flux_expression
        + c3_model.reactions.NADPHoxm_tx.flux_expression
    ),
    lb=0,
    ub=0,
)
c3_model.add_cons_vars(const)

# Rubisco
# ATP/NADPH 3:1 constraints
const = c3_model.problem.Constraint(
    3 * c3_model.reactions.RXN_961_p.flux_expression -
    c3_model.reactions.RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p.flux_expression, 
    lb=0,
    ub=0,
)
c3_model.add_cons_vars(const)

### Using more complex constrains

- Using `Constraint` objects
- Define expression and their bounds
- Can combine multiple reactions

Example: Sum of nitrate uptake in mesophyll and bundle sheath
cells cannot surpass a flux sum of 1000

In [19]:
# Skipped
# Constrains for light dependent maintenance costs
atp_b = c4_model.reactions.get_by_id("B_ATPase_tx")
photon_b = c4_model.reactions.get_by_id("B_Photon_tx")
atp_m = c4_model.reactions.get_by_id("M_ATPase_tx")
photon_m = c4_model.reactions.get_by_id("M_Photon_tx")

const_b = c4_model.problem.Constraint(
    (0.0049 * photon_b.flux_expression + 2.7852) - atp_b.flux_expression,
    lb=0,
    ub=0,
)

const_m = c4_model.problem.Constraint(
    (0.0049 * photon_m.flux_expression + 2.7852) - atp_m.flux_expression,
    lb=0,
    ub=0,
)
c4_model.add_cons_vars([const_b, const_m])

In [20]:
# CONSTRAINT : Total N uptake must not surpass defined upper bound
bs_nitrate = c4_model.reactions.get_by_id("B_Nitrate_tx").flux_expression
m_nitrate = c4_model.reactions.get_by_id("M_Nitrate_tx").flux_expression

nitrate_ratio = c4_model.problem.Constraint(
    bs_nitrate + m_nitrate, lb=0, ub=1000
)

c4_model.add_cons_vars([nitrate_ratio])

In [21]:
# Skipped
# ATP/NADPH 3:1 constraints
for cell in ["B_", "M_"]:
    const = c4_model.problem.Constraint(
        c4_model.reactions.get_by_id(f"{cell}ATPase_tx").flux_expression
        - 3
        * (
            c4_model.reactions.get_by_id(f"{cell}NADPHoxc_tx").flux_expression
            + c4_model.reactions.get_by_id(
                f"{cell}NADPHoxp_tx"
            ).flux_expression
            + c4_model.reactions.get_by_id(
                f"{cell}NADPHoxm_tx"
            ).flux_expression
        ),
        lb=0,
        ub=0,
    )
    c4_model.add_cons_vars(const)


### Simulating the model by optimising biomass production

$\rightarrow$ biomass reaction

Solution example:

In [22]:
c4_model.objective = "B_AraCore_Biomass_tx"
c4_model.optimize()

,fluxes,reduced_costs
M_PRO_PROTON_vc,0.000000e+00,9.787731e-18
M_Ca_tx,0.000000e+00,-0.000000e+00
M_H2O_xc,1.921197e-13,-0.000000e+00
M_sCIT_biomass,0.000000e+00,-0.000000e+00
M_ACETYLGLUTKIN_RXN_p,0.000000e+00,-2.602085e-18
...,...,...
B_SYRINGICACID_BIOSYNTHESIS_c,0.000000e+00,0.000000e+00
B_VANILLICACID_BIOSYNTHESIS_c,0.000000e+00,0.000000e+00
B_PYR_MAL_pc,1.060279e+02,0.000000e+00
MB_MAL_c,1.251705e+02,0.000000e+00


### C4 mechanism

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/C4_photosynthesis_NADP-ME_type_en.svg/1920px-C4_photosynthesis_NADP-ME_type_en.svg.png" alt="drawing" width="1500"/>
<font size="5">PEPC: PEP Carboxylase; NADP-MDH : Malate Dehydrogenase; NADP-ME: NADP Malic enzyme; PPDK: Pyruvate-phosphate dikinase.</font>
 


### C4 mechanism

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/C4_photosynthesis_NADP-ME_type_en.svg/1920px-C4_photosynthesis_NADP-ME_type_en.svg.png" alt="drawing" width="3000"/>
<font size="5">PEPC: PEP Carboxylase; NADP-MDH : Malate Dehydrogenase; NADP-ME: NADP Malic enzyme; PPDK: Pyruvate-phosphate dikinase.</font>
 


#### Malate dehydrogenase

In [23]:
for reaction in c4_model.reactions.query(
    "M_MALATE_DEHYDROGENASE_NADP_RXN_[c,p]"
):
    print(
        reaction.id,
        c4_model.optimize().fluxes[reaction.id]
    )

M_MALATE_DEHYDROGENASE_NADP_RXN_p 146.79216348544594


### C4 mechanism

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/C4_photosynthesis_NADP-ME_type_en.svg/1920px-C4_photosynthesis_NADP-ME_type_en.svg.png" alt="drawing" width="3000"/>
<font size="5">PEPC: PEP Carboxylase; NADP-MDH : Malate Dehydrogenase; NADP-ME: NADP Malic enzyme; PPDK: Pyruvate-phosphate dikinase.</font>
 


#### NADP malic enzyme

In [24]:
for reaction in c4_model.reactions.query(
    "B_MALIC_NADP_RXN_p"
):
    print(
        reaction.id,
        c4_model.optimize().fluxes[reaction.id]
    )

B_MALIC_NADP_RXN_p 118.00792044544465


### C4 mechanism

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/C4_photosynthesis_NADP-ME_type_en.svg/1920px-C4_photosynthesis_NADP-ME_type_en.svg.png" alt="drawing" width="3000"/>
<font size="5">PEPC: PEP Carboxylase; NADP-MDH : Malate Dehydrogenase; NADP-ME: NADP Malic enzyme; PPDK: Pyruvate-phosphate dikinase.</font>
 


#### Pyruvate-phosphate dikinase

In [25]:
for reaction in c4_model.reactions.query(
    "M_PYRUVATEORTHOPHOSPHATE_DIKINASE_RXN_p"
):
    print(
        reaction.id,
        c4_model.optimize().fluxes[reaction.id]
    )

M_PYRUVATEORTHOPHOSPHATE_DIKINASE_RXN_p 125.17047572434029


### C4 mechanism

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/C4_photosynthesis_NADP-ME_type_en.svg/1920px-C4_photosynthesis_NADP-ME_type_en.svg.png" alt="drawing" width="3000"/>
<font size="5">PEPC: PEP Carboxylase; NADP-MDH : Malate Dehydrogenase; NADP-ME: NADP Malic enzyme; PPDK: Pyruvate-phosphate dikinase.</font>
 


#### Transporters

In [26]:
for reaction in c4_model.reactions.query(
    "MB_MAL"
):
    print(
        reaction.id, reaction.reaction, 
        c4_model.optimize().fluxes[reaction.id]
    )

MB_MAL_c M_MAL_c <=> B_MAL_c 125.17047572434059


In [27]:
for reaction in c4_model.reactions.query(
    "MB_PYRUVATE"
):
    print(
        reaction.id, reaction.reaction,
        c4_model.optimize().fluxes[reaction.id])

MB_PYRUVATE_c M_PYRUVATE_c <=> B_PYRUVATE_c -125.1704757243403


### Comparison

In [28]:
c4_model.objective = "B_AraCore_Biomass_tx"
c4_solution = c4_model.optimize()

In [29]:
c3_model.objective = "AraCore_Biomass_tx"
c3_solution = c3_model.optimize()

#### Biomass production

In [30]:
print(c4_solution)
print(c3_solution)

<Solution 1.445 at 0x7f30411274f0>
<Solution 0.727 at 0x7f3025a89d90>


#### RuBisco reactions

In [31]:
for reaction in c4_model.reactions.query("RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN"):
    print(reaction.id,  c4_solution.fluxes[reaction.id])
for reaction in c4_model.reactions.query("RXN_961"):
    print(reaction.id,  c4_solution.fluxes[reaction.id])

M_RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p 0.0
B_RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p 135.7518212702515
M_RXN_961_p 0.0
B_RXN_961_p 0.0


In [32]:
for reaction in c3_model.reactions.query("RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN"):
    print(reaction.id,  c3_solution.fluxes[reaction.id])
for reaction in c3_model.reactions.query("RXN_961"):
    print(reaction.id,  c3_solution.fluxes[reaction.id])

RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p 72.60511972942481
RXN_961_p 24.20170657647494


#### CO2 intake and O2 output 

In [33]:
for reaction in c4_model.reactions.query("O2_tx"):
    print(reaction.id,  c4_solution.fluxes[reaction.id])

M_CO2_tx 125.17047572434018
M_O2_tx -62.5852378621703
B_CO2_tx 0.0
B_O2_tx -91.78093974262296


In [34]:
for reaction in c3_model.reactions.query("O2_tx"):
    print(reaction.id,  c3_solution.fluxes[reaction.id])

CO2_tx 63.015022739727875
O2_tx -77.71311993279774


### Questions?

### Thank you very much!!